# Create a csv to match our movie model

## movies.csv

In [61]:
# Transform and combine movies, links and ratings
# id/title/year/avg_rate/imdb_id/tmdb_id

In [62]:
import pandas as pd

In [63]:
df_movies = pd.read_csv('raw_data/movies.csv')

In [64]:
df_movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [65]:
# Create new column with the year

df_movies['year'] = df_movies['title'].str.extract(r'\((\d{4})\)')

In [66]:
# Remove year from title column

df_movies['title'] = df_movies['title'].replace(r'\((\d{4})\)', '', regex=True).str.strip()

In [67]:
# Drop genres, they wil be handled in a seperate notebook

df_movies = df_movies[['movieId','title', 'year']]

In [68]:
df_movies.head(5)

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


In [69]:
# To remove any parenthesized titles
# df_movies['title'] = df_movies['title'].str.replace(r'\(.*\)', '', regex=True)

df_movies.loc[:, 'title'] = df_movies['title'].str.replace(r'\(.*\)', '', regex=True)

In [70]:
# Examine movies with commas in the title
df_commas = df_movies[df_movies['title'].str.contains(',')]

## Address movies with a ',' in the title

In [71]:
df_commas.head()

,movieId,title,year
10,11,"American President, The",1995
28,29,"City of Lost Children, The",1995
39,40,"Cry, the Beloved Country",1995
49,50,"Usual Suspects, The",1995
53,54,"Big Green, The",1995


In [72]:
# Move articles to front and strip out trailing comma

def move_articles(title):
    articles = ['The', 'An', 'A', 'Les', 'L\'', 'Das', 'Le', 'El', 'La', 'Da']
    part = title.split(', ')

    # If there is more than one part, i.e., title was split by a comma
    if len(part) > 1:
        # Get the first word of the second part
        first_word = part[1].split(' ')[0]
        if first_word in articles:
            return f'{first_word} {part[0]}'

    return title

In [73]:
# Apply to the 'title' column
#df_movies['title'] = df_movies['title'].apply(move_articles)  

df_movies.loc[:, 'title'] = df_movies['title'].apply(move_articles)

In [74]:
# Examine the remaining movies with commas
df_commas = df_movies[df_movies['title'].str.contains(',')]

In [76]:
df_commas.tail()

,movieId,title,year
86246,287903,"New York Portrait, Chapter II",1981
86259,288007,"Film, the Living Record of Our Memory",2022
86316,288255,"God, the Universe and Everything Else",1988
86331,288289,"Ruby Gillman, Teenage Kraken",2023
86496,288855,"Weddings, christenings and funerals",2022


In [77]:
df_movies.head(15)

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995
5,6,Heat,1995
6,7,Sabrina,1995
7,8,Tom and Huck,1995
8,9,Sudden Death,1995
9,10,GoldenEye,1995


## links.csv - to add imdb and tmdb id columns to df_movies

In [78]:
df_link = pd.read_csv('raw_data/links.csv')

In [79]:
df_link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
86532,288967,14418234,845861.0
86533,288971,11162178,878958.0
86534,288975,70199,150392.0
86535,288977,23050520,1102551.0


In [80]:
df_link.isnull

<bound method DataFrame.isnull of        movieId    imdbId     tmdbId
0            1    114709      862.0
1            2    113497     8844.0
2            3    113228    15602.0
3            4    114885    31357.0
4            5    113041    11862.0
...        ...       ...        ...
86532   288967  14418234   845861.0
86533   288971  11162178   878958.0
86534   288975     70199   150392.0
86535   288977  23050520  1102551.0
86536   288983  11644948   940588.0

[86537 rows x 3 columns]>

In [81]:
# Fill null values with 0
df_link['tmdbId'] = df_link['tmdbId'].fillna(0)

In [82]:
# Extract IMDB column 
imdbId_col = df_link["imdbId"]

In [83]:
# Add the extracted column 
df_movies = pd.concat([df_movies, imdbId_col.rename("imdbId")], axis=1)

In [84]:
# Extract TMDB column 
tmdbId_col = df_link["tmdbId"]

In [85]:
# Add the extracted column 
df_movies = pd.concat([df_movies, tmdbId_col.rename("tmdbId")], axis=1)

In [86]:
df_movies

,movieId,title,year,imdbId,tmdbId
0,1,Toy Story,1995,114709,862.0
1,2,Jumanji,1995,113497,8844.0
2,3,Grumpier Old Men,1995,113228,15602.0
3,4,Waiting to Exhale,1995,114885,31357.0
4,5,Father of the Bride Part II,1995,113041,11862.0
...,...,...,...,...,...
86532,288967,State of Siege: Temple Attack,2021,14418234,845861.0
86533,288971,Ouija Japan,2021,11162178,878958.0
86534,288975,The Men Who Made the Movies: Howard Hawks,1973,70199,150392.0
86535,288977,Skinford: Death Sentence,2023,23050520,1102551.0


In [87]:
# Change TMDB to integer
df_movies['tmdbId'] = df_movies['tmdbId'].astype(int)

In [88]:
df_movies

,movieId,title,year,imdbId,tmdbId
0,1,Toy Story,1995,114709,862
1,2,Jumanji,1995,113497,8844
2,3,Grumpier Old Men,1995,113228,15602
3,4,Waiting to Exhale,1995,114885,31357
4,5,Father of the Bride Part II,1995,113041,11862
...,...,...,...,...,...
86532,288967,State of Siege: Temple Attack,2021,14418234,845861
86533,288971,Ouija Japan,2021,11162178,878958
86534,288975,The Men Who Made the Movies: Howard Hawks,1973,70199,150392
86535,288977,Skinford: Death Sentence,2023,23050520,1102551


## ratings.csv - to generate average and total rating

In [89]:
df_ratings = pd.read_csv('raw_data/ratings.csv')

In [90]:
df_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,1225734739
1,1,110,4.0,1225865086
2,1,158,4.0,1225733503
3,1,260,4.5,1225735204
4,1,356,5.0,1225735119
...,...,...,...,...
33832157,330975,8340,2.0,1091583256
33832158,330975,8493,2.5,1091585709
33832159,330975,8622,4.0,1091581777
33832160,330975,8665,3.0,1091581765


In [91]:
average_ratings = df_ratings.groupby('movieId')['rating'].mean().reset_index()

In [92]:
average_ratings

,movieId,rating
0,1,3.893508
1,2,3.278179
2,3,3.171271
3,4,2.868395
4,5,3.076957
...,...,...
83234,288967,3.500000
83235,288971,0.500000
83236,288975,4.000000
83237,288977,3.000000


In [93]:
df_movies = pd.merge(df_movies, average_ratings, how='outer',on='movieId')

In [94]:
avg_rating_count = df_ratings.groupby('movieId')['rating'].agg(['mean', 'count']).reset_index()

In [95]:
df_movies = pd.merge(df_movies, avg_rating_count, how='outer',on='movieId')

In [96]:
df_movies

,movieId,title,year,imdbId,tmdbId,rating,mean,count
0,1,Toy Story,1995,114709,862,3.893508,3.893508,76813.0
1,2,Jumanji,1995,113497,8844,3.278179,3.278179,30209.0
2,3,Grumpier Old Men,1995,113228,15602,3.171271,3.171271,15820.0
3,4,Waiting to Exhale,1995,114885,31357,2.868395,2.868395,3028.0
4,5,Father of the Bride Part II,1995,113041,11862,3.076957,3.076957,15801.0
...,...,...,...,...,...,...,...,...
86532,288967,State of Siege: Temple Attack,2021,14418234,845861,3.500000,3.500000,1.0
86533,288971,Ouija Japan,2021,11162178,878958,0.500000,0.500000,1.0
86534,288975,The Men Who Made the Movies: Howard Hawks,1973,70199,150392,4.000000,4.000000,1.0
86535,288977,Skinford: Death Sentence,2023,23050520,1102551,3.000000,3.000000,1.0


In [97]:
df_movies = df_movies.round({'rating': 1, 'mean': 1,})

In [98]:
df_movies


,movieId,title,year,imdbId,tmdbId,rating,mean,count
0,1,Toy Story,1995,114709,862,3.9,3.9,76813.0
1,2,Jumanji,1995,113497,8844,3.3,3.3,30209.0
2,3,Grumpier Old Men,1995,113228,15602,3.2,3.2,15820.0
3,4,Waiting to Exhale,1995,114885,31357,2.9,2.9,3028.0
4,5,Father of the Bride Part II,1995,113041,11862,3.1,3.1,15801.0
...,...,...,...,...,...,...,...,...
86532,288967,State of Siege: Temple Attack,2021,14418234,845861,3.5,3.5,1.0
86533,288971,Ouija Japan,2021,11162178,878958,0.5,0.5,1.0
86534,288975,The Men Who Made the Movies: Howard Hawks,1973,70199,150392,4.0,4.0,1.0
86535,288977,Skinford: Death Sentence,2023,23050520,1102551,3.0,3.0,1.0


In [99]:
# Reorder columns
df_movies = df_movies[['movieId', 'title', 'year', 'rating', 'imdbId', 'tmdbId', 'mean', 'count']]

In [100]:
# Rename columns to match database model
df_movies = df_movies.rename(columns={"movieId": "id", "rating": "avg_rate", "imdbId": "imdb_id", "tmdbId": "tmdb_id", "count": "total_ratings"})

In [101]:
df_movies = df_movies.fillna(0)

In [102]:
df_movies['total_ratings'] = df_movies['total_ratings'].astype(int)

In [103]:
df_movies

,id,title,year,avg_rate,imdb_id,tmdb_id,mean,total_ratings
0,1,Toy Story,1995,3.9,114709,862,3.9,76813
1,2,Jumanji,1995,3.3,113497,8844,3.3,30209
2,3,Grumpier Old Men,1995,3.2,113228,15602,3.2,15820
3,4,Waiting to Exhale,1995,2.9,114885,31357,2.9,3028
4,5,Father of the Bride Part II,1995,3.1,113041,11862,3.1,15801
...,...,...,...,...,...,...,...,...
86532,288967,State of Siege: Temple Attack,2021,3.5,14418234,845861,3.5,1
86533,288971,Ouija Japan,2021,0.5,11162178,878958,0.5,1
86534,288975,The Men Who Made the Movies: Howard Hawks,1973,4.0,70199,150392,4.0,1
86535,288977,Skinford: Death Sentence,2023,3.0,23050520,1102551,3.0,1


In [104]:
# Drop auto index and make movie id the index
df_movies.set_index('id', inplace=True)

In [105]:
df_movies

,title,year,avg_rate,imdb_id,tmdb_id,mean,total_ratings
id,,,,,,,
1,Toy Story,1995,3.9,114709,862,3.9,76813
2,Jumanji,1995,3.3,113497,8844,3.3,30209
3,Grumpier Old Men,1995,3.2,113228,15602,3.2,15820
4,Waiting to Exhale,1995,2.9,114885,31357,2.9,3028
5,Father of the Bride Part II,1995,3.1,113041,11862,3.1,15801
...,...,...,...,...,...,...,...
288967,State of Siege: Temple Attack,2021,3.5,14418234,845861,3.5,1
288971,Ouija Japan,2021,0.5,11162178,878958,0.5,1
288975,The Men Who Made the Movies: Howard Hawks,1973,4.0,70199,150392,4.0,1


In [106]:
df_movies.isnull().sum()

title            0
year             0
avg_rate         0
imdb_id          0
tmdb_id          0
mean             0
total_ratings    0
dtype: int64

In [46]:
# Uncomment if needed to write csv file
# df_movies.to_csv('DB_models/DB_movies1.csv')